In [2]:
import json
import numpy as np

In [ ]:
"""
PyTorch shizz.
"""

# from __future__ import unicode_literals, print_function, division
# from io import open
# import unicodedata
# import string
# import re
# import random

# import torch
# import torch.nn as nn
# from torch import optim
# import torch.nn.functional as F

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# class Lang:
#     def __init__(self, name):
#         self.name = name
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {0: "SOS", 1: "EOS"}
#         self.n_words = 2  # Count SOS and EOS

#     def addSentence(self, sentence):
#         for word in sentence.split(' '):
#             self.addWord(word)

#     def addWord(self, word):
#         if word not in self.word2index:
#             self.word2index[word] = self.n_words
#             self.word2count[word] = 1
#             self.index2word[self.n_words] = word
#             self.n_words += 1
#         else:
#             self.word2count[word] += 1
            
            
# def readLangs(lang1, lang2, reverse=False):
#     print("Reading lines...")

#     # Read the file and split into lines
#     lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
#         read().strip().split('\n')

#     # Split every line into pairs and normalize
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

#     # Reverse pairs, make Lang instances
#     if reverse:
#         pairs = [list(reversed(p)) for p in pairs]
#         input_lang = Lang(lang2)
#         output_lang = Lang(lang1)
#     else:
#         input_lang = Lang(lang1)
#         output_lang = Lang(lang2)

#     return input_lang, output_lang, pairs

In [3]:
def read_data(file_name):
    """
    Reads the data and returns it in a list.
    """
    
    f = open(file_name, "r")
    return [line.strip().split() for line in f.readlines()]


def get_dictionary(file_name):
    """
    Obtains the vocabulary of a file and returns it 
    in a dictionary to be able to use w2i.
    """
    
    file = open(file_name) 
    return json.load(file)


def reverse_dictionary(dictionary):
    """
    Reverses the dictionary such that i2w can be used.
    """
    
    reversed_dict = {}
    
    for word, index in dictionary.items():
        reversed_dict[index] = word
    return reversed_dict


def initialize_embedding(dictionary, dimension):
    """
    Initialize embeddings randomly using a uniform
    distribution. The embeddings are standardized
    such that the mean is zero and the standard deviation is 0.01.
    """
    
    print(len(dictionary))
    
    embedding = np.random.uniform(-0.1, 0.1, (len(dictionary), dimension))
    embedding = embedding - np.mean(embedding, axis=0)
    embedding = embedding/np.std(embedding)
    return embedding


def embed_sentence(sentence, embeddings, dictionary):
    """
    Returns the embedding of a sentence.
    """
    
    embedding = []
    
    for word in sentence:
        index = dictionary[word]
        embedding.append(embeddings[index, :])
    return embedding

train_english = read_data("data/train_preprocessed.en")
train_french = read_data("data/train_preprocessed.fr")
# get_pairs(train_english, train_french)


# w2i_french = get_dictionary("data/train_preprocessed.fr.json")
# w2i_english = get_dictionary("data/train_preprocessed.en.json")
# i2w_french = reverse_dictionary(w2i_french)
# i2w_english = reverse_dictionary(w2i_english)

# embeddings = initialize_embedding(french_dict, 50)
# embed_sentence(train_english[0], embeddings, english_dict)

29000
29000
